In [1]:
import pandas as pd
from tqdm import tqdm

## Add Slogan column to Clusters Alarms

In [5]:
cluster_filtered = pd.read_parquet("real-time clusters/20230101-20240101_real_time_clusters_filtered.parquet")

In [2]:
adsl = pd.read_parquet("alarms datasets/tx/20230101-20240101_inpas_tx_preprocess__adsl__last_event__last_event__ext1.parquet")
pdh = pd.read_parquet("alarms datasets/tx/20230101-20240101_inpas_tx_preprocess__pdh__last_event__last_event__ext1.parquet")
sdh = pd.read_parquet("alarms datasets/tx/20230101-20240101_inpas_tx_preprocess__sdh__last_event__last_event__ext1.parquet")
ptn = pd.read_parquet("alarms datasets/tx/20230101-20240101_inpas_tx_preprocess__ptn__last_event__last_event__ext1.parquet")

In [4]:
mob = pd.read_parquet("alarms datasets/mob/20230101-20240101_inpas_mob_preprocess__an__last_event__last_event__ext1.parquet")

In [5]:
mob["link_type"] = "None"

In [7]:
mob = mob[["alarm_id", "std_probable_cause_no", "link_type"]]
adsl = adsl[["alarm_id", "std_probable_cause_no", "link_type"]]
pdh = pdh[["alarm_id", "std_probable_cause_no", "link_type"]]
sdh = sdh[["alarm_id", "std_probable_cause_no", "link_type"]]
ptn = ptn[["alarm_id", "std_probable_cause_no", "link_type"]]

In [8]:
all_alarms = pd.concat([adsl, pdh, sdh, ptn, mob])

In [11]:
all_alarms["link_type"] = all_alarms["link_type"].fillna("None")

In [13]:
all_alarms["link_type"].unique()

array(['l12', 'None', 'otc', 'lfd', 'at1', 'dmx2', 'ps3', 'ps4', 'ps',
       'nd', 'ad3', 'gb8', 'gb1', 'go1', 'l10', 'gbe', 'l100', '/0-',
       'itl', 'nq0', '30', 'lim', 'sqp', 'n34', 'rs-', 'nq', 's3t', 'nm',
       '482', 'ubt', 'n80', 'f-m', 'prn', '3-s', 'f-t', 'res', 'qp',
       '480', 'sdh', 'ed', 's1t', 'nsq', 'sc1', 'sc4', 'f-i', 'ode',
       '2rm', 'n8', 'px1', 'qp0', 'dlo', 'sc3', '120', 'ala', '0pv',
       'dcn', 'dc2', 'erv', 'fm', 'bb-', '3ll', 'ds1', 'dm1', 'tim',
       's3r', '3s1', 'gen', '300', 's1r', 'eri', '.e.', 'cdz', 'om',
       'dtv', 'ela', 'dvr', 't-i', '.o.', 'cs0', 't-t', 'dbz', '3rm',
       '.tx', 'rio', 'dcf', 'nm0', '100', 'ond', 'fm6', 'n2', 'dm-', 'll',
       'sit', 'ali', 'cam', 'sub', '3li', '1pv', 'dz', 'qp2', '132',
       'os2', 'emp', '0', '0vr', '16w', 'ibe', 'k9', 'n20', '301', 'px',
       'cdn', '3-c', '5-c', '5-s', '1-c', '1-s', 'ene', '-ca', '-ss',
       '2-s', '0-s', '8-s', '6-s', '9-s', '7-s', '6-c', '8-c', '0-c',
       '2-c',

In [3]:
clusters_with_slogans = pd.merge(cluster_filtered, all_alarms, on="alarm_id", how="left")

NameError: name 'cluster_filtered' is not defined

In [3]:
clusters_with_slogans.to_parquet("real-time clusters/20230101-20240101_real_time_clusters_filtered_with_slogans.parquet")

NameError: name 'clusters_with_slogans' is not defined

## Create Bitmap

In [3]:
clusters_with_slogans = pd.read_parquet("real-time clusters/20230101-20240101_real_time_clusters_filtered_with_slogans.parquet")

In [9]:
clusters_with_slogans

,cluster_id2,alarm_id,std_probable_cause_no,ne_type,link_type,ne_type_slogan_link
0,202301010000_16,COS1P;136885076,apparato isolato,ad16,None,ad16_apparato isolato_None
1,202301010000_16,COS1P;136885115,apparato isolato,ad16,None,ad16_apparato isolato_None
2,202301010000_25,COS1P;136884642,apparato isolato,adm4,None,adm4_apparato isolato_None
3,202301010000_25,COS1P;136884882,apparato isolato,ad16,None,ad16_apparato isolato_None
4,202301010000_25,COS1P;136884883,apparato isolato,ad16,None,ad16_apparato isolato_None
...,...,...,...,...,...,...
8128873,202312312354_7,EFM;6591f07dadefac0001744e96,base station connectivity problem,vnbts,None,vnbts_base station connectivity problem_None
8128874,202312312354_7,EFM;6591f08739628d00013784db,base station connectivity problem,vnbts,None,vnbts_base station connectivity problem_None
8128875,202312312354_7,EFM;6591f08cadefac0001744ea1,base station connectivity problem,vnbts,None,vnbts_base station connectivity problem_None
8128876,202312312340_13,EFMTX;6591ee0801021b00010facb4,fermo totale apparato,adsl,l12,adsl_fermo totale apparato_l12


In [4]:
clusters_with_slogans = clusters_with_slogans[["cluster_id2", "alarm_id", "std_probable_cause_no", "ne_type", "link_type"]]

In [5]:
clusters_with_slogans["link_type"] = clusters_with_slogans["link_type"].fillna("None")

In [10]:
len(clusters_with_slogans["link_type"].unique())

91

In [7]:
clusters_with_slogans['ne_type_slogan_link'] = clusters_with_slogans.apply(lambda row: str(row.ne_type) + '_' + str(row.std_probable_cause_no) + '_' + str(row.link_type), axis=1)

In [11]:
clusters_with_slogans

,cluster_id2,alarm_id,std_probable_cause_no,ne_type,link_type,ne_type_slogan_link
0,202301010000_16,COS1P;136885076,apparato isolato,ad16,None,ad16_apparato isolato_None
1,202301010000_16,COS1P;136885115,apparato isolato,ad16,None,ad16_apparato isolato_None
2,202301010000_25,COS1P;136884642,apparato isolato,adm4,None,adm4_apparato isolato_None
3,202301010000_25,COS1P;136884882,apparato isolato,ad16,None,ad16_apparato isolato_None
4,202301010000_25,COS1P;136884883,apparato isolato,ad16,None,ad16_apparato isolato_None
...,...,...,...,...,...,...
8128873,202312312354_7,EFM;6591f07dadefac0001744e96,base station connectivity problem,vnbts,None,vnbts_base station connectivity problem_None
8128874,202312312354_7,EFM;6591f08739628d00013784db,base station connectivity problem,vnbts,None,vnbts_base station connectivity problem_None
8128875,202312312354_7,EFM;6591f08cadefac0001744ea1,base station connectivity problem,vnbts,None,vnbts_base station connectivity problem_None
8128876,202312312340_13,EFMTX;6591ee0801021b00010facb4,fermo totale apparato,adsl,l12,adsl_fermo totale apparato_l12


In [12]:
grouped_clusters = clusters_with_slogans.groupby("cluster_id2")
unique_ne_type_slogans_link = clusters_with_slogans['ne_type_slogan_link'].unique()

In [13]:
unique_ne_type_slogans_link

array(['ad16_apparato isolato_None', 'adm4_apparato isolato_None',
       'lhbts_fermo parziale sito tec_None', ...,
       'oadm1_apparato isolato_None', 'rmx5_loss of signal los_None',
       'adm1_sef intermittente sef_pre'], dtype=object)

In [14]:
len(unique_ne_type_slogans_link)

1755

In [18]:
# Definisci una funzione per calcolare la presenza di ciascuno slogan
def calculate_presence(group):
    presence = {slogan: 0 for slogan in unique_ne_type_slogans_link}
    for slogan in group['ne_type_slogan_link']:
        presence[slogan] = 1
    presence['cluster_id2'] = group['cluster_id2'].iloc[0]
    return pd.Series(presence)

tqdm.pandas()

# Converti l'oggetto DataFrameGroupBy in una lista di gruppi
grouped_clusters_list = [group for name, group in grouped_clusters]

# Calcola l'indice per ogni quarto della lista
quarter_idx = len(grouped_clusters_list) // 4

# Dividi la lista in quattro parti
grouped_clusters_list_1 = grouped_clusters_list[:quarter_idx]
grouped_clusters_list_2 = grouped_clusters_list[quarter_idx:2*quarter_idx]
grouped_clusters_list_3 = grouped_clusters_list[2*quarter_idx:3*quarter_idx]
grouped_clusters_list_4 = grouped_clusters_list[3*quarter_idx:]

In [24]:
# Applica la funzione a ciascuna parte e crea un DataFrame
result_df_1 = pd.concat([calculate_presence(group) for group in tqdm(grouped_clusters_list_1)], axis=1).T
result_df_1 = result_df_1.set_index('cluster_id2')
result_df_1.to_parquet("result_df_1.parquet")

100%|██████████| 156234/156234 [02:13<00:00, 1166.19it/s]


In [25]:
result_df_2 = pd.concat([calculate_presence(group) for group in tqdm(grouped_clusters_list_2)], axis=1).T
result_df_2 = result_df_2.set_index('cluster_id2')
result_df_2.to_parquet("result_df_2.parquet")

  0%|          | 0/156234 [00:00<?, ?it/s]

100%|██████████| 156234/156234 [02:31<00:00, 1029.94it/s]


In [26]:
result_df_3 = pd.concat([calculate_presence(group) for group in tqdm(grouped_clusters_list_3)], axis=1).T
result_df_3 = result_df_3.set_index('cluster_id2')
result_df_3.to_parquet("result_df_3.parquet")

100%|██████████| 156234/156234 [02:33<00:00, 1017.88it/s]


In [27]:
result_df_4 = pd.concat([calculate_presence(group) for group in tqdm(grouped_clusters_list_4)], axis=1).T
result_df_4 = result_df_4.set_index('cluster_id2')
result_df_4.to_parquet("result_df_4.parquet")

100%|██████████| 156235/156235 [02:46<00:00, 936.97it/s] 


In [28]:
result_df_1 = pd.read_parquet("result_df_1.parquet")
result_df_2 = pd.read_parquet("result_df_2.parquet")
result_df_3 = pd.read_parquet("result_df_3.parquet")
result_df_4 = pd.read_parquet("result_df_4.parquet")
# Unisci i risultati in un unico DataFrame
result_df = pd.concat([result_df_1, result_df_2, result_df_3, result_df_4])

In [25]:
# Rinomina la colonna 'None' in 'None' come stringa
result_df = result_df.rename(columns={None: 'None'})

In [29]:
result_df.to_parquet("real-time clusters/result_df_gt_2_ne_type_link.parquet")